# Laboratorio 8

## Integrantes

### Sergio Orellana - 221122

### Andre Marroquin - 22266

### Rodrigo Mansilla - 22611

# Link del repositorio

https://github.com/mar22266/LABORATORIOS-IA.git


# Task 1 Teoría


1.  Investigar el algoritmo AC-3 y su relación con el algoritmo de backtracking search

        R// El algoritmo AC-3 (Arc Consistency Algorithm 3) es un método utilizado en problemas de satisfacción de restricciones (CSP) para lograr la consistencia de arcos. Su función principal es reducir los dominios de las variables eliminando valores que no satisfacen las restricciones binarias entre pares de variables. Esto se realiza iterativamente hasta que no se puedan hacer más reducciones o se detecte que algún dominio queda vacío, indicando que el CSP no tiene solución.

        Por otro lado, el backtracking search es una técnica de búsqueda en profundidad que asigna valores a las variables de manera tentativa y retrocede backtracks cuando se encuentra una inconsistencia, explorando así todas las posibles combinaciones hasta encontrar una solución o determinar que no existe.

        La relación entre AC-3 y el backtracking search esta en que AC-3 puede usarse como un paso previo al backtracking para reducir los dominios de las variables, facilitando así la búsqueda. Al aplicar AC-3 antes de iniciar el backtracking, se eliminan valores inconsistentes, lo que puede disminuir significativamente el espacio de búsqueda y mejorar la eficiencia del proceso de

2.  Defina en sus propias palabras el término “Arc Consistency”

        R// La "consistencia de arcos" es una propiedad que aparece en problemas de satisfacción de restricciones. Esto implica que, cuando se tienen dos variables relacionadas por una restricción, cada valor posible de una variable debe tener al menos un valor compatible en la otra. En otras palabras, no deberían existir valores en el dominio de una variable que, al ser elegidos, impidan encontrar valores válidos en la otra variable según la restricción que las une.


# Task 2 - CSP con Backtracking, Beam y Local Search


### 1. Definición del problema, variables y restricciones

Las variables que se utilizarán son:

- Exam1
- Exam2
- Exam3
- Exam4
- Exam5
- Exam6
- Exam7

Los dominios de cada variable son los 3 días que se nos indicaron:

- Lunes
- Martes
- Miércoles

Restricciones

En la forma que tomaremos la ejecución de cada examen es la siguiente:

- Rodri tomará Exam1, Exam2 y Exam3
- Suarino tomará Exam2, Exam4 y Exam5
- Sergio tomará Exam3, Exam5 y Exam6
- Andre tomará Exam1, Exam4 y Exam7


In [ ]:
import time

# Definición de variables 
exams = ["Exam1", "Exam2", "Exam3", "Exam4", "Exam5", "Exam6", "Exam7"]
days = ["Lunes", "Martes", "Miércoles"]

# estudiantes inscritos en cada examen
enrollment = {
    "Exam1": ["Rodri", "Andre"],
    "Exam2": ["Rodri", "Suarino"],
    "Exam3": ["Rodri", "Sergio"],
    "Exam4": ["Suarino", "Andre"],
    "Exam5": ["Suarino", "Sergio"],
    "Exam6": ["Sergio"],
    "Exam7": ["Andre"],
}


# funcion que encuentra los conflictos
def count_conflicts(assignment):
    conflicts = 0
    # Se comparan pares de examnes
    for i in range(len(exams)):
        for j in range(i + 1, len(exams)):
            exam1 = exams[i]
            exam2 = exams[j]
            if exam1 in assignment and exam2 in assignment:
                if assignment[exam1] == assignment[exam2]:
                    # Si comparten estudiante y estan en el mismo dia hay conflicto
                    if set(enrollment[exam1]).intersection(enrollment[exam2]):
                        conflicts += 1
    return conflicts


# funcion que verifica si agregar exam, day es consistente con la asignacion actual
def is_consistent(assignment, exam, day):
    for other_exam, assigned_day in assignment.items():
        if assigned_day == day:
            # Si 2 exam comparten estudiante, no pueden ser el mismo dia
            if set(enrollment[exam]).intersection(enrollment[other_exam]):
                return False
    return True

Las siguientes funciones servirán para poder ver que los estudiantes cumplen con la reestriccion que todos los exámenes tienen que hacerse en días diferentes, ningún estudiante tenga más de un examen por día y que los estudiantes que toman el mismo curso no tengan exámenes el mismo día.


In [ ]:
def check_solution(assignment):
    # se verifica que cada examene este asignado a un dia valido
    for exam, day in assignment.items():
        if day not in days:
            return False, f"El examen {exam} se asignó a un día inválido: {day}"

    # Para cada estudiante, se construye su horario y se verifica que no tenga mas de un examen por dia
    student_schedule = {}
    for exam, day in assignment.items():
        for student in enrollment[exam]:
            if student not in student_schedule:
                student_schedule[student] = []
            student_schedule[student].append((exam, day))

    for student, exam_days in student_schedule.items():
        day_count = {}
        for exam, day in exam_days:
            day_count[day] = day_count.get(day, 0) + 1
        for day, count in day_count.items():
            if count > 1:
                return (
                    False,
                    f"El estudiante {student} tiene {count} exámenes el día {day}.",
                )

    return True, "La solución cumple todas las restricciones."


def display_student_schedule(assignment):
    # Se construye un diccionario que mapea cada estudiante 
    student_schedule = {}
    for exam, day in assignment.items():
        for student in enrollment[exam]:
            student_schedule.setdefault(student, []).append((exam, day))

    # Se imprime el horario de cada estudiante
    print("Horarios de exámenes por estudiante:")
    for student, exam_days in student_schedule.items():
        print(f"Estudiante {student}:")
        day_order = {"Lunes": 1, "Martes": 2, "Miércoles": 3}
        exam_days.sort(key=lambda x: day_order[x[1]])
        for exam, day in exam_days:
            print(f"  - {exam}: {day}")
        print()

### Backtracking Search


In [ ]:
def backtracking_search(assignment={}):
    # asigancion completa como caso base
    if len(assignment) == len(exams):
        return assignment

    # se selecciona la siguiente variable sin asignar
    unassigned = [exam for exam in exams if exam not in assignment]
    exam = unassigned[0]

    for day in days:
        if is_consistent(assignment, exam, day):
            assignment[exam] = day
            result = backtracking_search(assignment)
            if result is not None:
                return result
            # Si no se encontro solución se deshace la asignacion
            del assignment[exam]
    return None

# saca los tiempos
start_bt = time.time()
solucion_bt = backtracking_search()
end_bt = time.time()
time_bt = end_bt - start_bt

print("Tiempo de ejecución:", time_bt, "segundos")
print("Solución:", solucion_bt)
valido, mensaje = check_solution(solucion_bt)
print("Verificación:", mensaje)
display_student_schedule(solucion_bt)

Tiempo de ejecución: 0.0010187625885009766 segundos
Solución: {'Exam1': 'Lunes', 'Exam2': 'Martes', 'Exam3': 'Miércoles', 'Exam4': 'Miércoles', 'Exam5': 'Lunes', 'Exam6': 'Martes', 'Exam7': 'Martes'}
Verificación: La solución cumple todas las restricciones.
Horarios de exámenes por estudiante:
Estudiante Rodri:
  - Exam1: Lunes
  - Exam2: Martes
  - Exam3: Miércoles

Estudiante Andre:
  - Exam1: Lunes
  - Exam7: Martes
  - Exam4: Miércoles

Estudiante Suarino:
  - Exam5: Lunes
  - Exam2: Martes
  - Exam4: Miércoles

Estudiante Sergio:
  - Exam5: Lunes
  - Exam6: Martes
  - Exam3: Miércoles



### Beam Search


In [ ]:
def beam_search(beam_width=3):
    # asignación vacia inicialmente
    beam = [{}]

    while beam:
        new_beam = []
        for assignment in beam:
            # Si la asignación es completa y sin conflictos se devuelve la solucion
            if len(assignment) == len(exams) and count_conflicts(assignment) == 0:
                return assignment

            # Seleccionar el siguiente examen a asignar
            unassigned = [exam for exam in exams if exam not in assignment]
            if not unassigned:
                continue
            next_exam = unassigned[0]
            for day in days:
                new_assignment = assignment.copy()
                new_assignment[next_exam] = day
                # se evalua la heuristica
                conflicts = count_conflicts(new_assignment)
                new_beam.append((conflicts, new_assignment))

        # se ordenan las asignacions por el num de conflictos y se agarran las mejores
        new_beam.sort(key=lambda x: x[0])
        beam = [assignment for conflicts, assignment in new_beam[:beam_width]]
    return None


start_beam = time.time()
solucion_beam = beam_search(beam_width=3)
end_beam = time.time()
time_beam = end_beam - start_beam

print("Tiempo de ejecución:", time_beam, "segundos")
print("Solución:", solucion_beam)
valido, mensaje = check_solution(solucion_beam)
print("Verificación:", mensaje)
display_student_schedule(solucion_beam)

Tiempo de ejecución: 0.0009999275207519531 segundos
Solución: {'Exam1': 'Lunes', 'Exam2': 'Martes', 'Exam3': 'Miércoles', 'Exam4': 'Miércoles', 'Exam5': 'Lunes', 'Exam6': 'Martes', 'Exam7': 'Martes'}
Verificación: La solución cumple todas las restricciones.
Horarios de exámenes por estudiante:
Estudiante Rodri:
  - Exam1: Lunes
  - Exam2: Martes
  - Exam3: Miércoles

Estudiante Andre:
  - Exam1: Lunes
  - Exam7: Martes
  - Exam4: Miércoles

Estudiante Suarino:
  - Exam5: Lunes
  - Exam2: Martes
  - Exam4: Miércoles

Estudiante Sergio:
  - Exam5: Lunes
  - Exam6: Martes
  - Exam3: Miércoles



### Local Search


In [ ]:
import random


# Genera una asigancion aleatoria
def random_assignment():
    return {exam: random.choice(days) for exam in exams}


def local_search(max_iterations=1000):
    current = random_assignment()

    for _ in range(max_iterations):
        current_conflicts = count_conflicts(current)
        if current_conflicts == 0:
            return current

        neighbors = []
        # Para cada examen se prueba cambiar el dia por cada opcsion distidiferente 
        for exam in exams:
            for day in days:
                if current[exam] != day:
                    neighbor = current.copy()
                    neighbor[exam] = day
                    neighbors.append((count_conflicts(neighbor), neighbor))

        # Se selecciona el vecino que minimiza los conflictos
        neighbors.sort(key=lambda x: x[0])
        best_conflict, best_neighbor = neighbors[0]

        # Si se mejora, se actualiza la solución actual
        if best_conflict < current_conflicts:
            current = best_neighbor
        else:
            # Si no hay mejora, se selecciona aleatoriamente un vecino
            current = random.choice(neighbors)[1]

    return current  


interations = 1000

start_local = time.time()
solucion_local = local_search(max_iterations=interations)
end_local = time.time()
time_local = end_local - start_local

print("Tiempo de ejecución:", time_local, "segundos")
print("Solución:", solucion_local)
valido, mensaje = check_solution(solucion_local)
print("Verificación:", mensaje)
display_student_schedule(solucion_local)

Tiempo de ejecución: 0.003000974655151367 segundos
Solución: {'Exam1': 'Lunes', 'Exam2': 'Martes', 'Exam3': 'Miércoles', 'Exam4': 'Miércoles', 'Exam5': 'Lunes', 'Exam6': 'Martes', 'Exam7': 'Martes'}
Verificación: La solución cumple todas las restricciones.
Horarios de exámenes por estudiante:
Estudiante Rodri:
  - Exam1: Lunes
  - Exam2: Martes
  - Exam3: Miércoles

Estudiante Andre:
  - Exam1: Lunes
  - Exam7: Martes
  - Exam4: Miércoles

Estudiante Suarino:
  - Exam5: Lunes
  - Exam2: Martes
  - Exam4: Miércoles

Estudiante Sergio:
  - Exam5: Lunes
  - Exam6: Martes
  - Exam3: Miércoles



### Conclusiones

En nuestros experimentos con el problema de programación de exámenes, implementamos tres algoritmos de satisfacción de restricciones: Backtracking, Beam Search y Local Search. Comparando tanto los tiempos de ejecución como las soluciones obtenidas, hemos observado lo siguiente:

- _Backtracking Search:_

  - Tardó 0.0010187625885009766 segundos en la ejecución.
  - Esta solución cumple con todas las restricciones, asignando días distintos para cada examen y garantizando que ningún estudiante tenga más de un examen en un mismo día. Este método es completo y, en este caso, mostró un rendimiento muy rápido.

- _Beam Search:_

  - Tardó 0.0009999275207519531 segundos en la ejecución
  - Esta solución obtenida mediante Beam Search también cumple con las restricciones. Este enfoque, que limita la búsqueda a las asignaciones parciales más prometedoras, obtuvo un tiempo de ejecución similar al de Backtracking. Aunque en problemas más complejos la elección del ancho del haz y la heurística podrían influir significativamente, en este caso el rendimiento fue excelente.

- _Local Search:_
  - Tardó 0.003000974655151367 segundos en la ejecución
  - El algoritmo Local Search encontró una solución válida mediante la mejora iterativa de una asignación aleatoria. Su tiempo de ejecución fue ligeramente superior al de los otros métodos, probablemente debido a la exploración de los vecinos en cada iteración, pero sigue siendo muy eficiente para este problema de tamaño reducido.

Los tres algoritmos lograron encontrar soluciones válidas que cumplen con las restricciones del problema de este laboratorio en tiempos de ejecución muy bajos. Nosotros consideramos que, para este caso en particular, cualquier método es adecuado. Sin embargo, en problemas de mayor tamaño, complejidad o mayor cantidad de estudiantes y exámenes, la elección del algoritmo dependerá de si se prioriza la garantía de completitud (como en Backtracking), la eficiencia en la reducción del espacio de búsqueda (Beam Search) o la rapidez en la mejora iterativa de una solución inicial (Local Search).

Los algoritmos tardaron diferentes tiempos por la forma en que exploran las posibles soluciones. Aunque todos fueron rápidos, Local Search fue un poco más lento porque parte de una solución aleatoria y necesita mejorarla paso a paso evaluando múltiples vecinos en cada iteración. En cambio, Backtracking y Beam Search fueron más directos, Backtracking explora de forma sistemática y completa, mientras que Beam Search reduce el espacio de búsqueda concentrándose en las mejores opciones.

# Referencias:

- GeeksforGeeks. (2024, 12 junio). Explain the Concept of Backtracking Search and Its Role in Finding Solutions to CSPs. GeeksforGeeks. https://www.geeksforgeeks.org/explain-the-concept-of-backtracking-search-and-its-role-in-finding-solutions-to-csps/

- Boris. (2025, 28 febrero). ARC consistency explained. BorisTheBrave.Com. https://www.boristhebrave.com/2021/08/30/arc-consistency-explained/

- AC-3 algorithm and backtracking. (s. f.). Stack Overflow. https://stackoverflow.com/questions/15462253/ac-3-algorithm-and-backtracking

- Lecture 3 - CSCI E-80. (s. f.). https://cs50.harvard.edu/extension/ai/2021/fall/notes/3/

- Arc consistency. (2005). https://www.cs.cmu.edu/afs/cs/project/jair/pub/volume24/samaras05a-html/node4.html

- Bowen, E. (2024, 19 noviembre). Understanding the beam search algorithm in NLP. Telnyx. https://telnyx.com/learn-ai/beam-search-algorithm

- Achury, N. F. C. (2024, 1 febrero). ¿Qué es y para qué sirven los algoritmos Backtracking o Retroceso?\*. Datum Digital Marketing. https://datumdigitalmarketing.com/noticias/que-es-y-para-que-sirven-los-algoritmos-backtracking-o-retroceso

- Building, C. (s. f.). Local search. https://www.complexica.com/narrow-ai-glossary/local-search
